In [11]:
%%capture [--no-stdout]
!pip install fhir-aggregator-client --no-cache-dir --quiet

In [12]:
%%capture [--no-stdout]
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(connected=True)
import itables.options as opt

opt.classes="display nowrap compact"
opt.buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
opt.maxBytes=0

%env FHIR_BASE= https://google-fhir.fhir-aggregator.org
!fq vocabulary vocabulary.tsv --fhir-base-url $FHIR_BASE
df = pd.read_csv('vocabulary.tsv', sep='\t').fillna('')

# Vocabulary filtering examples
It's easy enough to get a straightforward table response - just utilize Pandas to filter appropriately to your interest. Some examples below:

## Exploring a study
Finding the vocabulary specific study is easy - filter according to the desired identifier in the `research_study_identifiers` column.

In [13]:
scratch_df = df[df['research_study_identifiers'].str.contains('1KG')]
scratch_df

,research_study_identifiers,path,documentation,code,display,system,extension_url,count,low,high,url,research_study_title,research_study_description,observation,research_study
3740,1KG,ServiceRequest.category,https://hl7.org/fhir/R4B/servicerequest-defini...,108252007,Laboratory procedure,http://snomed.info/sct,,1.0,,,https://google-fhir.fhir-aggregator.org/Servic...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3741,1KG,ServiceRequest.code,https://hl7.org/fhir/R4B/servicerequest-defini...,15220000,Laboratory test,http://snomed.info/sct,,1.0,,,https://google-fhir.fhir-aggregator.org/Servic...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3742,1KG,DocumentReference.type,https://hl7.org/fhir/R4B/documentreference-def...,VCF,VCF,https://ftp.1000genomes.ebi.ac.uk/data_format,,46.0,,,https://google-fhir.fhir-aggregator.org/Docume...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3743,1KG,DocumentReference.type,https://hl7.org/fhir/R4B/documentreference-def...,NEW,NEW,https://ftp.1000genomes.ebi.ac.uk/data_format,,2.0,,,https://google-fhir.fhir-aggregator.org/Docume...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3744,1KG,DocumentReference.category,https://hl7.org/fhir/R4B/documentreference-def...,1,Chromosome 1,https://ftp.1000genomes.ebi.ac.uk/chromosome,,2.0,,,https://google-fhir.fhir-aggregator.org/Docume...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,1KG,Patient.extension,https://hl7.org/fhir/R4B/patient-definitions.h...,LWK,LWK,,https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...,116.0,,,https://google-fhir.fhir-aggregator.org/Patien...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3822,1KG,Patient.extension,https://hl7.org/fhir/R4B/patient-definitions.h...,ASW,ASW,,https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...,112.0,,,https://google-fhir.fhir-aggregator.org/Patien...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3823,1KG,Patient.extension,https://hl7.org/fhir/R4B/patient-definitions.h...,MXL,MXL,,https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...,107.0,,,https://google-fhir.fhir-aggregator.org/Patien...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...
3824,1KG,Patient.extension,https://hl7.org/fhir/R4B/patient-definitions.h...,TSI,TSI,,https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...,112.0,,,https://google-fhir.fhir-aggregator.org/Patien...,1000 Genomes Project Sample Metadata,,Observation/db717d0f-1ca1-5c2f-b01c-17ee8c73fbf6,ResearchStudy/4502d1f5-5275-5be7-9942-21f7fb8a...


# Exploring available conditions
Or query the vocab table for the available condition codes of the various patients in each research study.

In [14]:
scratch_df = df[df['path'].str.contains('Condition.code')]
scratch_df

,research_study_identifiers,path,documentation,code,display,system,extension_url,count,low,high,url,research_study_title,research_study_description,observation,research_study
11,ICGC-LUCA_KR,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,C15.5,C15.5,https://terminology.hl7.org/5.1.0/NamingSystem...,,400.0,,,https://google-fhir.fhir-aggregator.org/Condit...,,ICGC DCC & Argo Personalised Genomic Character...,Observation/4fc6bcaa-bb07-51e9-99b2-a85f679d81ab,ResearchStudy/92f949c7-4244-5218-8569-59707365...
230,TNP SARDANA,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,Mucous adenocarcinoma,Mucous adenocarcinoma,https://data.humantumoratlas.org,,1.0,,,https://google-fhir.fhir-aggregator.org/Condit...,,,Observation/e6d6c1dc-d227-5b29-ab6f-3c4102589475,ResearchStudy/b7015858-983c-5e51-bcd6-af788edd...
231,TNP SARDANA,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,Not Reported,Not Reported,https://data.humantumoratlas.org,,1.0,,,https://google-fhir.fhir-aggregator.org/Condit...,,,Observation/e6d6c1dc-d227-5b29-ab6f-3c4102589475,ResearchStudy/b7015858-983c-5e51-bcd6-af788edd...
534,HTAPP,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,Lobular and ductal carcinoma,Lobular and ductal carcinoma,https://data.humantumoratlas.org,,9.0,,,https://google-fhir.fhir-aggregator.org/Condit...,,,Observation/165e6f12-8c34-5cec-9964-57222cc93b42,ResearchStudy/64eafc92-6a65-5fb6-9d0e-15778b9a...
535,HTAPP,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,Ductal carcinoma NOS,Ductal carcinoma NOS,https://data.humantumoratlas.org,,43.0,,,https://google-fhir.fhir-aggregator.org/Condit...,,,Observation/165e6f12-8c34-5cec-9964-57222cc93b42,ResearchStudy/64eafc92-6a65-5fb6-9d0e-15778b9a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21671,HCMI-CMDC,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,Serous surface papillary carcinoma,Serous surface papillary carcinoma,https://gdc.cancer.gov/primary_diagnosis,,1.0,,,https://google-fhir.fhir-aggregator.org/Condit...,NCI Cancer Model Development for the Human Can...,,Observation/7314f7c1-5afb-5722-9293-8c97cfe9f0ea,ResearchStudy/e2d5a499-d435-5e37-93ef-0a39dfcb...
21672,HCMI-CMDC,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,"Carcinosarcoma, NOS","Carcinosarcoma, NOS",https://gdc.cancer.gov/primary_diagnosis,,1.0,,,https://google-fhir.fhir-aggregator.org/Condit...,NCI Cancer Model Development for the Human Can...,,Observation/7314f7c1-5afb-5722-9293-8c97cfe9f0ea,ResearchStudy/e2d5a499-d435-5e37-93ef-0a39dfcb...
21673,HCMI-CMDC,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,"Serous cystadenocarcinoma, NOS","Serous cystadenocarcinoma, NOS",https://gdc.cancer.gov/primary_diagnosis,,1.0,,,https://google-fhir.fhir-aggregator.org/Condit...,NCI Cancer Model Development for the Human Can...,,Observation/7314f7c1-5afb-5722-9293-8c97cfe9f0ea,ResearchStudy/e2d5a499-d435-5e37-93ef-0a39dfcb...
21674,HCMI-CMDC,Condition.code,https://hl7.org/fhir/R4B/condition-definitions...,Abdominal fibromatosis,Abdominal fibromatosis,https://gdc.cancer.gov/primary_diagnosis,,1.0,,,https://google-fhir.fhir-aggregator.org/Condit...,NCI Cancer Model Development for the Human Can...,,Observation/7314f7c1-5afb-5722-9293-8c97cfe9f0ea,ResearchStudy/e2d5a499-d435-5e37-93ef-0a39dfcb...
